In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
)

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader

def custom_loader(file_path: str):
    if file_path.endswith(".pdf"):
        return PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        return TextLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

loader = DirectoryLoader("personal", glob="**/*", show_progress=True, loader_cls=custom_loader)
docs = loader.load()

In [ ]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.2",
)

In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "personal" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
# vector_store.add_documents(docs)

In [ ]:
stats = index.describe_index_stats()

In [ ]:
stats

In [ ]:
vector_store.similarity_search("gre", k=4)

In [ ]:
doc_retriever = vector_store.as_retriever()

tool for pdf reader

In [ ]:
# from pypdf import PdfReader
# from langchain_core.tools import tool

# @tool
# def marks_reader(pdf_path: str):
#     """
#     Read the marks of subjects from the pdf and return subject marks
#     """
#     pdf_reader = PdfReader(pdf_path)
#     text = ""
    
#     for page_num in range(len(pdf_reader.pages)):
#         text += pdf_reader.pages[page_num].extract_text()
    
#     return text

text mark reader tool

In [ ]:
from langchain_core.tools import tool

@tool
def marks_reader(self, file_path: str) -> str:
    """Read content from a text file."""
    try:
        if not os.path.exists(file_path):
            return f"Error: File not found at path {file_path}"
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"

In [ ]:
tools = [marks_reader]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools.invoke("can you tell me my first year marks using the markreader tool?")

In [ ]:
from langchain.prompts import PromptTemplate

template = """
    You are a helpful assistant that can provide information based on both vector search from a database and other tools.

    ### Task:
    You will be provided with a user query. Your goal is to respond to the query by doing the following:
    1. Retrieve relevant information from the vector database (Pinecone) that best matches the query.
    2. If additional context or information is required that can be retrieved from tools (such as reading a PDF), use the appropriate tool.
    3. Combine the information from the database and tools to provide a well-structured and complete response.

    ### User Query:
    {user_query}

    ### Relevant Documents (from Pinecone vector database):
    {retrieved_documents}

    ### Tool Usage:
    If the relevant documents or context are not sufficient to answer the query, use the tools you have. For example, if the user is asking for marks of certain subjects, use the marks_reader tool to extract information from the document.
    If you used any tools, describe how the tool was used in your response.

    ### Answer:
    Your answer should be concise, clear, and comprehensive, using the retrieved documents and any tool-assisted information.
 """

prompt = PromptTemplate(
    input_variables=["user_query", "retrieved_documents"],
    template=template,
)
print(prompt)

In [ ]:
chain = prompt | llm_with_tools
print(chain)

In [ ]:
user_query = input("Your query :: \n")
retrieved_documents = doc_retriever.invoke(user_query)

response = chain.invoke({"user_query": user_query, "retrieved_documents": retrieved_documents})
print(response.content)

## let's try with deepseek model

In [ ]:
from langchain_ollama import ChatOllama

deepseek = ChatOllama(
    model="deepseek-r1",
    temperature=0,
)

In [ ]:
deepseek.invoke("which model are you?")

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

In [ ]:
embeddings

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
# vector_store.add_documents(docs)

In [ ]:
# index_ids

In [ ]:
vector_store.similarity_search("statement of purpose", k=1)

In [ ]:
retriever_deepseek = vector_store.as_retriever(k=3)

In [ ]:
from langchain_core.tools import tool

@tool
def marks_reader(self, file_path: str) -> str:
    """Read content from a text file."""
    try:
        if not os.path.exists(file_path):
            return f"Error: File not found at path {file_path}"
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"
    
tools = [marks_reader]
deepseek_with_tools = deepseek.bind_tools(tools)

In [ ]:
from langchain.prompts import PromptTemplate

template = """
    You are a helpful assistant that can provide information based on both vector search from a database and other tools.

    ### Task:
    You will be provided with a user query. Your goal is to respond to the query by doing the following:
    1. Retrieve relevant information from the vector database (Pinecone) that best matches the query.
    2. If additional context or information is required that can be retrieved from tools (such as reading a PDF), use the appropriate tool.
    3. Combine the information from the database and tools to provide a well-structured and complete response.

    ### User Query:
    {user_query}

    ### Relevant Documents (from Pinecone vector database):
    {retrieved_documents}

    ### Tool Usage:
    If the relevant documents or context are not sufficient to answer the query, use the tools you have. For example, if the user is asking for marks of certain subjects, use the marks_reader tool to extract information from the document.
    If you used any tools, describe how the tool was used in your response.

    ### Answer:
    Your answer should be concise, clear, and comprehensive, using the retrieved documents and any tool-assisted information.
 """

prompt = PromptTemplate(
    input_variables=["user_query", "retrieved_documents"],
    template=template,
)

In [ ]:
chain = prompt | deepseek
print(chain)

In [ ]:
user_query = input("Your query :: \n")
retrieved_documents = retriever_deepseek.invoke(user_query)

response = chain.invoke({"user_query": user_query, "retrieved_documents": retrieved_documents})
print(response.content)

let's try with mistral model

In [ ]:
from langchain_ollama import ChatOllama

mistral = ChatOllama(
    model="mistral",
    temperature=0,
)

In [ ]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mistral",
)

In [ ]:
result = embeddings.embed_query("hello")

In [ ]:
len(result)

In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "personal" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
indexes = vector_store.add_documents(docs)

In [ ]:
# index.delete(delete_all=True)

In [ ]:
len(indexes)

In [ ]:
vector_store.similarity_search("NALA group")

In [ ]:
mistral_retriever = vector_store.as_retriever(search_kwargs={'k':2})

In [ ]:
mistral_retriever.invoke("statement of purpose")

In [ ]:
from langchain_core.tools import tool

@tool
def marks_reader(self, file_path: str="nirajan_transcript.txt") -> str:
    """Read content from a text file."""
    try:
        if not os.path.exists(file_path):
            return f"Error: File not found at path {file_path}"
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"
    
tools = [marks_reader]
mistral_with_tools = mistral.bind_tools(tools)

In [ ]:
from langchain.prompts import PromptTemplate

template = """
    You are a helpful assistant that can provide information based on both vector search from a database and other tools.

    ### Task:
    You will be provided with a user query. Your goal is to respond to the query by doing the following:
    1. Retrieve relevant information from the vector database (Pinecone) that best matches the query.
    2. If additional context or information is required that can be retrieved from tools (such as reading a text file for marks), use the appropriate tool.
    3. Combine the information from the database and tools to provide a well-structured and complete response.

    ### User Query:
    {user_query}

    ### Relevant Documents (from Pinecone vector database):
    {retrieved_documents}

    ### Tool Usage:
    If the relevant documents or context are not sufficient to answer the query, use the tools you have. For example, if the user is asking for marks of certain subjects, use the marks_reader tool to extract information from the document.
    If you used any tools, describe how the tool was used in your response.

    ### Answer:
    Your answer should be concise, clear, and comprehensive, using the retrieved documents and any tool-assisted information.
 """

prompt = PromptTemplate(
    input_variables=["user_query", "retrieved_documents"],
    template=template,
)

In [ ]:
chain = prompt | mistral_with_tools

In [ ]:
user_query = input("Your query :: \n")
retrieved_documents = mistral_retriever.invoke(user_query)

response = chain.invoke({"user_query": user_query, "retrieved_documents": retrieved_documents})
print(response.content)